In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
raw_survey_data = pd.read_excel("../data/01_raw/DCSurveyRaw240205.xlsx")

In [ ]:
# Exclude indicated columns

selected_survey_data = raw_survey_data.drop(columns=["ID", "Nazwa", "Czas ostatniej modyfikacji", "Adres e-mail", "Godzina rozpoczęcia", "Godzina ukończenia"])

In [ ]:
selected_survey_data.head()

In [ ]:
#make list of object type columns

object_columns = selected_survey_data.select_dtypes(include=['object']).columns

#manually keep columns that are useful

usefull_categories = ["Czy kiedykolwiek programowałeś/aś?",
                      "Czy kiedykolwiek używałeś/aś narzędzi klasy Business Intelligence (np. Power BI, Tableau)",
                      "Czy działasz lub działałeś/aś w organizacji studenckiej lub innej zorganizowanej społeczności?",
                      "Czy masz doświadczenie w mentoringu lub uczeniu innych?",
                      """Liczba godzin tygodniowo, które jesteś przeciętnie w stanie wygospodarować na działalność w Data Community (działalność "z domu")""",
                      """Spotkania "roboczo-organizacyjne" preferuję odbywać w formie:""",
                      "Płeć",
                      "Tryb studiów",
                      "Jestem aktywny/a zawodowo",
                      "Pracuję w branży IT/DATA"]

In [ ]:
object_columns

In [ ]:
#make lisst of numeric columns

numeric_columns = selected_survey_data.select_dtypes(include=['int64', 'float64']).columns.to_list()
numeric_columns

In [ ]:
# select only numeric columns

numeric_survey_data = selected_survey_data.select_dtypes(include=['number'])

# calculate correlation matrix

correlation_matrix = numeric_survey_data.corr(method='spearman')

# plot correlation matrix

plt.figure(figsize=(20, 20))

sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')

# plt.show()

In [ ]:
#calculate means for each column

means = numeric_survey_data.mean().sort_values(ascending=False)
means

In [ ]:
cols_for_analysis = usefull_categories + numeric_columns

In [ ]:
# write to csv file dataframe with 'cols_for_analysis' columns

selected_survey_data[cols_for_analysis].to_csv("../data/01_raw/SelectedSurveySata.csv", index=False)

In [ ]:

selected_survey_data = selected_survey_data[selected_survey_data["Płeć"] != "Inna"]

In [ ]:
import numpy as np

# Prepare list for results
results = []

# Iterate over all combinations of selected columns
for col_cat in usefull_categories:
    for col_num in numeric_columns:

        # calulate mean for each column without groupping

        mean = selected_survey_data[col_num].mean()

        # grpuping by category and calculating mean for each group
        groups = selected_survey_data.groupby(col_cat)[col_num].mean()

        # Calculating difference between max and min value for each group
        max_min_diff = groups.max() - groups.min()

        # Appending results to the list
        results.append((col_cat, col_num, max_min_diff, groups.max(), groups.min(), mean))


# store results in a dataframe
results_df = pd.DataFrame(results, columns=["Category", "Column", "MaxMinDiff", "Max", "Min", "Mean"])

# Sort results by MaxMinDiff in descending order

results_sorted = results_df.sort_values(by="MaxMinDiff", ascending=False)

# Wyświetlenie pierwszych 10 wyników
results_sorted


In [ ]:
# For each 'Category' and 'Column' from 'results_sorted' dataframe plot barplot with 'Category' on x-axis and 'Column' on y-axis
for i, row in results_sorted[:20].iterrows():


# show 'Category' as a title on the top of the plot
    plt.title(row["Category"])

    # order categories on plot by mean value descending

    selected_survey_data.groupby(row["Category"])[row["Column"]].mean().sort_values(ascending=False).index

    # order categories on plot by mean value descending and keep consistency in colors of categories

    plt.xlim(0, 5.5)
    

    sns.barplot(data=selected_survey_data, x=row["Column"], y=row["Category"], ci=None, order=selected_survey_data.groupby(row["Category"])[row["Column"]].mean().sort_values(ascending=False).index)


    # remove y-axis label
    plt.ylabel("")

    # add space after title

    plt.title(row["Category"], pad=20)
    plt.show()
